In [4]:
pwd

'/export/home/ingole/My Files/Projects/Text-Synthesis-and-Compression-Project/Text-Synthesis-and-Compression-Project'

In [2]:
import os

In [3]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.textSynthesisAndCompression.constants import *
from src.textSynthesisAndCompression.utils.commom import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        #self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from src.textSynthesisAndCompression.logging import logger
from src.textSynthesisAndCompression.utils.commom import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-07 09:56:07,735: INFO: commom: yaml file: config/config.yaml loaded successfully]
[2024-01-07 09:56:07,738: INFO: commom: Created directory at: artifacts]
[2024-01-07 09:56:07,740: INFO: commom: Created directory at: artifacts/data_ingestion]
[2024-01-07 09:56:09,141: INFO: 2374601659: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 4106490
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e24652a166b4d3ee11c777eb1f97f6838e777d764bcae7777d3ad1dccd0d3c07"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FE32:0B3C:282332F:315BCA4:659A7535
Accept-Ranges: bytes
Date: Sun, 07 Jan 2024 09:56:08 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdfw8210146-DFW
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1704621368.123073,VS0,VE441
Vary: Authorization